<a href="https://colab.research.google.com/github/AharonRemez/LLM---Project/blob/main/LLM%20-%20Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install tiktoken
!pip install datasets

In [3]:
import shutil

url1 = ("/content/drive/MyDrive/LLM/gpt_model.py")
filename1 = url1.split('/')[-1]
url2 = ("/content/drive/MyDrive/LLM/weights_GPT.py")
filename2 = url2.split('/')[-1]
url3 = ("/content/drive/MyDrive/LLM/process_training.py")
filename3 = url3.split('/')[-1]


shutil.copy(url1, filename1)
shutil.copy(url2, filename2)
shutil.copy(url3, filename3)

'process_training.py'

In [5]:
from datasets import load_dataset

data = load_dataset("avaliev/chat_doctor")
train_dataset = data['train']
train_val = data['validation']
train_test = data['test']

print("train_dataset:", len(train_dataset))
print("train_val:", len(train_val))
print("train_test:", len(train_test))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train_dataset: 95588
train_val: 11949
train_test: 11949


**Format Alpace - Change to this format of the Q&A structure**

In [6]:
def format_input(entry):
 instruction_text = (
    f"Below is an instruction that describes a task. "
    f"Write a response that appropriately completes the request."
    f"\n\n### Instruction:\n{entry['instruction']}"
 )

 input_text = (
    f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
 )
 return instruction_text + input_text

**Check the structure of questions:**

In [7]:
model_input = format_input(train_dataset[50])
desired_response = f"\n\n### Response:\n{train_dataset[50]['output']}"
print(model_input + desired_response)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
If you are a doctor, please answer the medical questions based on the patient's description.

### Input:
My 3 year old son has developed red, blotchy areas on his skin. It began on his cheeks and around his mouth, and I wondered if he d eaten something he was allergic to?? Within a few days, it had spread to his triceps, and forearms, upper thighs, and lower legs. The red areas are generally warm to the touch, but there is no raised bump or roughness, rather smooth skin. It doesn t itch or hurt. He has a history of eczema (fairly mild), but this is clearly something different. Otherwise he is a healthy kid.

### Response:
Hello, Thank you for asking at Chat Doctor. I went through your sons history and would like to make suggestions for him as follows:1. From your provided history, I would first think of atomic dermatitis, though it should itch intensely, which is 

**Split the data into train, test and validation**

In [8]:
train_data = train_dataset
test_data = train_test
val_data = train_val

print("Training set length:", len(train_data))
print("Validation set length:", len(val_data))
print("Test set length:", len(test_data))

Training set length: 95588
Validation set length: 11949
Test set length: 11949


**Mplementing an instruction dataset class**

In [9]:
from torch.utils.data import Dataset

class InstructionDataset(Dataset):
  def __init__(self, data, tokenizer):
    self.data = data
    self.encoded_texts = []
    for entry in data:
      instruction_plus_input = format_input(entry)
      response_text = f"\n\n### Response:\n{entry['output']}"
      full_text = instruction_plus_input + response_text
      self.encoded_texts.append(tokenizer.encode(full_text))

  def __getitem__(self, index):
    return self.encoded_texts[index]

  def __len__(self):
    return len(self.data)

**Import the tokenizer**

In [10]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
#print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

In [11]:
import torch

def custom_collate_fn(batch, pad_token_id=50256, ignore_index=-100, device="cpu", allowed_max_length=None):
    batch_max_length = max(len(item)+1 for item in batch)   # קובע את האורך של הרצף הארוך ביותר בקבוצה (מוסיף 1 עבור טוקן סוף המשפט)
    inputs_lst, targets_lst = [], []   # רשימות לאחסון הקלטים והמטרות עבור כל דוגמה בקבוצה
    for item in batch:   # לולאה על כל דוגמה בקבוצה
        new_item = item.copy()   # יוצרים עותק של הדוגמה כדי לא לשנות את המקור
        new_item += [pad_token_id]   # מוסיפים טוקן סוף טקסט לדוגמה (כדי לציין את סוף הטקסט)

        padded = (new_item + [pad_token_id] * (batch_max_length - len(new_item)))   # מרפדים את הדוגמה כך שתתאים לאורך הרצף הארוך ביותר בקבוצה
        inputs = torch.tensor(padded[:-1])   # קלטים: כל הדוגמה חוץ מהטוקן האחרון (סוף הטקסט)
        targets = torch.tensor(padded[1:])   # מטרות: מתחילים מהטוקן השני ועד הסוף (כולל סוף הטקסט)

        mask = targets == pad_token_id   # בודקים איפה מופיעים טוקני הריפוד במטרות
        indices = torch.nonzero(mask).squeeze()   # מוצאים את המיקומים של טוקני הריפוד
        if indices.numel() > 1:   # אם יש יותר מטוקן ריפוד אחד במטרות
            targets[indices[1:]] = ignore_index   # כל טוקני הריפוד מעבר לראשון מוחלפים בערך ההתעלמות (-100)

        if allowed_max_length is not None:   # אם יש מגבלה על אורך מקסימלי
            inputs = inputs[:allowed_max_length]   # חותכים את הקלט לפי האורך המקסימלי המותר
            targets = targets[:allowed_max_length]   # חותכים את המטרות לפי האורך המקסימלי המותר

        inputs_lst.append(inputs)   # מוסיפים את הקלט לרשימת הקלטים
        targets_lst.append(targets)   # מוסיפים את המטרות לרשימת המטרות

    inputs_tensor = torch.stack(inputs_lst).to(device)   # מאחדים את כל הקלטים למבנה של טנזור
    targets_tensor = torch.stack(targets_lst).to(device)   # מאחדים את כל המטרות למבנה של טנזור
    return inputs_tensor, targets_tensor   # מחזירים את טנזורי הקלט והמטרות

**Example:**

In [12]:
inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]
batch = (inputs_1, inputs_2, inputs_3)
inputs, targets = custom_collate_fn(batch)
print(inputs)
print(targets)

tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]])
tensor([[    1,     2,     3,     4, 50256],
        [    6, 50256,  -100,  -100,  -100],
        [    8,     9, 50256,  -100,  -100]])


**Creating Dataloders:**

In [13]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print("Device:", device)

In [14]:
from functools import partial
customized_collate_fn = partial(custom_collate_fn, allowed_max_length=1024) #device=device,

In [15]:
from torch.utils.data import DataLoader

num_workers = 0
batch_size = 8

torch.manual_seed(123)

train_dataset = InstructionDataset(train_data, tokenizer)
train_loader = DataLoader(
  train_dataset,
  batch_size=batch_size,
  collate_fn=customized_collate_fn,
  shuffle=True,
  drop_last=True,
  num_workers=num_workers
)
val_dataset = InstructionDataset(val_data, tokenizer)
val_loader = DataLoader(
  val_dataset,
  batch_size=batch_size,
  collate_fn=customized_collate_fn,
  shuffle=False,
  drop_last=False,
  num_workers=num_workers
)
test_dataset = InstructionDataset(test_data, tokenizer)
test_loader = DataLoader(
  test_dataset,
  batch_size=batch_size,
  collate_fn=customized_collate_fn,
  shuffle=False,
  drop_last=False,
  num_workers=num_workers
)

**Stage of importing and applying parameters into the model:**

In [16]:
import urllib.request
url = (
"https://raw.githubusercontent.com/rasbt/"
"LLMs-from-scratch/main/ch05/"
"01_main-chapter-code/gpt_download.py"
)
filename = url.split('/')[-1]
urllib.request.urlretrieve(url, filename)

('gpt_download.py', <http.client.HTTPMessage at 0x7921cc1cae00>)

In [17]:
from gpt_download import download_and_load_gpt2
from gpt_model import GPTModel
from weights_GPT import load_weights_into_gpt

BASE_CONFIG = {
 "vocab_size": 50257, # Vocabulary size
 "context_length": 1024, # Context length
 "drop_rate": 0.0, # Dropout rate
 "qkv_bias": True # Query-key-value bias
}
model_configs = {
 "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
 "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
 "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
 "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}
CHOOSE_MODEL = "gpt2-medium (355M)"
BASE_CONFIG.update(model_configs[CHOOSE_MODEL])
model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
settings, params = download_and_load_gpt2(
 model_size=model_size,
 models_dir="gpt2"
)
model = GPTModel(BASE_CONFIG)
load_weights_into_gpt(model, params)
model.eval();

File already exists and is up-to-date: gpt2/355M/checkpoint
File already exists and is up-to-date: gpt2/355M/encoder.json
File already exists and is up-to-date: gpt2/355M/hparams.json
File already exists and is up-to-date: gpt2/355M/model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2/355M/model.ckpt.index
File already exists and is up-to-date: gpt2/355M/model.ckpt.meta
File already exists and is up-to-date: gpt2/355M/vocab.bpe


**check if all work clearly:**

In [18]:
torch.manual_seed(123)
input_text = format_input(val_data[0])
print(input_text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
If you are a doctor, please answer the medical questions based on the patient's description.

### Input:
I would like to know more about fluid in the lungs - this happened after a bowel obstruction that did let go after 4 days. I also have lupus and felt that possibly I had a flare at the same time, and inflammation of the lungs. Have also had a 3 bi-pass in 2012.


In [19]:
from process_training import generate, text_to_token_ids, token_ids_to_text

token_ids = generate(
 model=model,
 idx=text_to_token_ids(input_text, tokenizer),
 max_new_tokens=35,
 context_size=BASE_CONFIG["context_length"],
 eos_id=50256,
)
generated_text = token_ids_to_text(token_ids, tokenizer)

In [20]:
from process_training import calc_loss_loader, train_model_simple

#model.to(device)
torch.manual_seed(123)
with torch.no_grad():
 train_loss = calc_loss_loader(
 train_loader, model, num_batches=5   # device
 )
 val_loss = calc_loss_loader(
 val_loader, model, num_batches=5   # device
)
print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Training loss: 3.55898175239563
Validation loss: 3.6469301700592043


**Training:**

In [21]:
import time

start_time = time.time()
torch.manual_seed(123)
optimizer = torch.optim.AdamW(
 model.parameters(), lr=0.00005, weight_decay=0.1
)
num_epochs = 3
train_losses, val_losses, tokens_seen = train_model_simple(
 model, train_loader, val_loader, optimizer,   # device
 num_epochs=num_epochs, eval_freq=5, eval_iter=5,
 start_context=format_input(val_data[0]), tokenizer=tokenizer
)
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train loss 3.351, Val loss 3.374
Ep 1 (Step 000005): Train loss 2.888, Val loss 2.909
Ep 1 (Step 000010): Train loss 2.852, Val loss 2.822
Ep 1 (Step 000015): Train loss 2.880, Val loss 2.793
Ep 1 (Step 000020): Train loss 2.739, Val loss 2.777
Ep 1 (Step 000025): Train loss 2.638, Val loss 2.759
Ep 1 (Step 000030): Train loss 2.708, Val loss 2.740
Ep 1 (Step 000035): Train loss 2.880, Val loss 2.726
Ep 1 (Step 000040): Train loss 2.726, Val loss 2.714
Ep 1 (Step 000045): Train loss 2.676, Val loss 2.699
Ep 1 (Step 000050): Train loss 2.667, Val loss 2.689
Ep 1 (Step 000055): Train loss 2.731, Val loss 2.683
Ep 1 (Step 000060): Train loss 2.627, Val loss 2.675
Ep 1 (Step 000065): Train loss 2.515, Val loss 2.665
Ep 1 (Step 000070): Train loss 2.651, Val loss 2.656
Ep 1 (Step 000075): Train loss 2.578, Val loss 2.648
Ep 1 (Step 000080): Train loss 2.722, Val loss 2.645
Ep 1 (Step 000085): Train loss 2.600, Val loss 2.642
Ep 1 (Step 000090): Train loss 2.837, Val loss

KeyboardInterrupt: 

**Download the model:**

In [30]:
import re
file_name = f"{re.sub(r'[ ()]', '', CHOOSE_MODEL) }-sft.pth"
torch.save(model.state_dict(), file_name)
print(f"Model saved as {file_name}")

Model saved as gpt2-medium355M-sft.pth


**Import the model:**

In [41]:
import torch
from gpt_model import GPTModel

model = GPTModel(BASE_CONFIG)

model.load_state_dict(torch.load("/content/gpt2-medium355M-sft.pth"))
model.eval()

<ipython-input-41-3cb50214b4d7>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/gpt2-medium355M-sft.pth"))


GPTModel(
  (tok_emb): Embedding(50257, 1024)
  (pos_emb): Embedding(1024, 1024)
  (drop_emb): Dropout(p=0.0, inplace=False)
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=1024, out_features=1024, bias=True)
        (W_key): Linear(in_features=1024, out_features=1024, bias=True)
        (W_value): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=1024, out_features=4096, bias=True)
          (1): GELU()
          (2): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): 

**Question asked:**

In [64]:
def ask_model(model, tokenizer, base_config, question, max_new_tokens=256):
  """
  Asks a question to the model and returns the generated response.

  Args:
    model: The GPT model.
    tokenizer: The tokenizer used for the model.
    base_config: The model's configuration.
    question: The question to ask the model.
    max_new_tokens: The maximum number of tokens to generate.

  Returns:
    The generated response from the model.
  """

  input_text = (
      f"Below is an instruction that describes a task. "
      f"Write a response that appropriately completes the request."
      f"\n\n### Instruction:\n{question}"
  )

  token_ids = generate(
      model=model,
      idx=text_to_token_ids(input_text, tokenizer),
      max_new_tokens=max_new_tokens,
      context_size=base_config["context_length"],
      eos_id=50256
  )

  generated_text = token_ids_to_text(token_ids, tokenizer)

  response_text = (
      generated_text[len(input_text):]
      .replace("### Response:", "")
      .strip()
  )

  return response_text

def main():
  # Example usage:
  question = "Hi Doctor, I am a male and I have had a stomachache for a few days now because of the pain I have laxatives a lot. What should I do?"

  response = ask_model(model, tokenizer, BASE_CONFIG, question)

  print(f"Question: {question}")
  print(f"Response: {response}")

if __name__ == "__main__":
  main()


Question: Hi Doctor, I am a male and I have had a stomachache for a few days now because of the pain I have laxatives a lot. What should I do?
Response: Hi, thanks for asking query to ChatDoctorI tested blood sugar with enzymatic tests, high newplasma Ig output and gastric ultrasound blood PRESSURE. I am very sorry for your complaint because you were suffering from diabetes which in case could be due to diabetes condition. I would like to suggest you i.t intravenous folic acid twice daily also. You can rectally lactate every day. I will suggest following X-ray study in case not much progress with digestive structure. A. Consult physician and get done some gastrostomatoscopy study. INSERT PIN laser biopsy done. FN want to know if both of you are affected by diarrhea. For this I suggest you to complete Dulbezos and resist before considering treatment. Thank you.
